# Задания

## Задание 1

Рассмотрим маленькую коллекцию документов, состоящую из следующих документов: 

“new york times”, “new york post”, “los angeles times”.

Перечислите все уникальные термы в коллекции и запишите матрицу терм-документ.

In [9]:
# Матрица терм-документ: 

import pandas as pd

columns = ['term','document']
docs = ['d1', 'd2', 'd3']

d1_data = [{'term': 'new', 'd1': 1, 'd2':1, 'd3':0},
         {'term': 'york',  'd1': 1, 'd2':1, 'd3':0},
         {'term': 'times',  'd1': 1, 'd2':0, 'd3':1},
          {'term': 'post',  'd1': 0, 'd2':1, 'd3':0},
           {'term': 'los',  'd1': 0, 'd2':0, 'd3':1},
           {'term': 'angeles',  'd1': 0, 'd2':0, 'd3':1}]
d1 = pd.DataFrame(data = d1_data)
d1

,d1,d2,d3,term
0,1,1,0,new
1,1,1,0,york
2,1,0,1,times
3,0,1,0,post
4,0,0,1,los
5,0,0,1,angeles


Найдите для всех терм частоты (tf) и обратные документные частоты idf.


Пусть дан запрос q=“new new times”. Найдите координаты вектора-запроса в нашем векторном пространстве и примените к нему tf − idf преобразование.

Используя косинусную меру близости в качестве функции релевантности, отсортируйте документы по убыванию релевантности запроса.

Найдем косинусные меры близости для всех трех документов с вектором q по формуле:

$cos(d_i, d_j) = \frac{d_1 \times d_2}{ ᛁᛁd_1ᛁᛁ ᛁᛁd_2ᛁᛁ} $

$d_1$ = (0.135, 0.135, 0.135)

$d_2$ = (0.135, 0.135, 0.366) 

$d_1$ = (0.366, 0.366, 0.135)


In [37]:
import numpy as np

d1 = np.array([0.135, 0.135, 0.135]) # new york times
d2 = np.array([0.135, 0.135, 0.366]) # new york post
d3 = np.array([0.366, 0.366, 0.135]) # los angeles times
q = np.array([0.27, 0.27, 0.135])

def cos_score(d, q):
    return np.dot(d,q)/(np.sqrt(np.dot(q,q))*np.sqrt(np.dot(d,d)))

print(cos_score(d1,q), cos_score(d2,q), cos_score(d3,q), sep='\n')

0.962250448649
0.731584510603
0.996414868109


Документы в порядке убывания релевантности запроса q: $d_3, d_1, d_2$

## Задание 2

Группа лингвистов проанализировала толковые и иноязычные словари русского языка и посчитала для каждой пары слов A, B из заданного списка слов следующий показатель: сколько раз слово B (или его иноязычный аналог) используется в определении слова A. Результаты приведены в таблице ниже.

![title](table.png)

Предположите, какие два слова больше всего похожи друг на друга по смыслу.


Выполните PPMI преобразование матрицы близости и посчитайте попарную близость между словами. Подтверждается ли ваша гипотеза из первого задания?


In [95]:
F = np.array([[0,16,13,12,10,6,0],
              [18,0,9,12,5,6,2],
             [37,7,0,6,7,4,0],
             [26,26,9,0,6,8,2],
             [39,11,22,27,0,3,0],
             [1,0,2,0,1,0,3],
              [4,1,4,1,4,3,0]])

words = ['работа','деятельность','труд','действие','дело','создание','творчество']

PPMI_matrix = np.zeros((7,7))
res = []
for i in range (len(words)):
    p_i = np.sum(F, axis=1)[i]/np.sum(F)
    for j in range (len(words)):
        if (i != j):
            p_j = np.sum(F, axis=0)[j]/np.sum(F)
            p_ij = F[i][j]/np.sum(F, axis=1)[i]
            PPMI = max(0, np.log2(p_ij/(p_i*p_j)))
            #print(words[i], words[j], PPMI)
            res.append((PPMI, words[i], words[j]))
            PPMI_matrix[i][j] = PPMI
            
sorted(res, key=lambda v: v[0], reverse=True)[:10]

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in log2


[(10.248961375058819, 'создание', 'творчество'),
 (6.5887107470334261, 'создание', 'труд'),
 (6.4269596770368143, 'создание', 'дело'),
 (5.8667438386513435, 'творчество', 'дело'),
 (5.5892098631224343, 'творчество', 'создание'),
 (5.0284949086479553, 'творчество', 'труд'),
 (4.5055695117331807, 'создание', 'работа'),
 (3.9453536733477095, 'творчество', 'работа'),
 (3.8778292821706875, 'деятельность', 'творчество'),
 (3.6978175877099009, 'работа', 'дело')]

In [82]:
print(PPMI_matrix)

[[  0.           3.48954627   3.23808028   3.14726512   3.69781759
    3.09835552   0.        ]
 [  2.88932492   0.           2.97246616   3.41216571   2.96271818
    3.36325611   3.87782928]
 [  3.46825805   2.10120655   0.           1.95157047   2.98754977
    2.31769837   0.        ]
 [  2.28714599   3.32219294   1.83977251   0.           2.09305894
    2.64559996   2.74513564]
 [  2.06083089   1.26990724   2.31800153   2.63811946   0.           0.41928486
    0.        ]
 [  4.50556951   0.           6.58871075   0.           6.42695968   0.
   10.24896138]
 [  3.94535367   2.98040062   5.02849491   3.05315696   5.86674384
    5.58920986   0.        ]]


Используйте SVD для снижения размерности и пересчитайте попарную близость между векторами слов. Подтверждается ли ваша гипотеза из первого задания?

In [104]:
U, s, V = np.linalg.svd(PPMI_matrix)

# обнулим последние три значения, k = 3
for i in range(4,7):
    s[i] = 0
    V[i] = np.zeros(7)
    for j in range(7):
        U[j][i] = 0


new_PPMI = U.dot(np.diag(s)).dot(V)
new_PPMI

array([[  0.32889984,   3.06844196,   3.82262413,   2.53498644,
          3.56422031,   3.33695553,  -0.42609659],
       [  3.6166534 ,   0.17836378,   3.54945203,   3.22296468,
          3.17303969,   2.29171058,   3.14191074],
       [  3.50385595,   1.95307587,   0.39792841,   1.52719699,
          2.3248292 ,   3.03726463,   0.09764437],
       [  2.36193078,   2.52985295,   1.68602938,  -0.18402949,
          3.39380756,   2.20749604,   2.0724109 ],
       [  1.30292344,   0.04979316,   1.61114927,   2.47027362,
          1.0403564 ,   1.45547654,   0.10594463],
       [  4.3191423 ,   0.03329731,   6.51493654,   0.01477574,
          6.12886482,   0.41508831,  10.52813685],
       [  3.59875138,   3.90566636,   4.45314207,   3.87625721,
          5.44023165,   5.35789186,   0.75155691]])

In [105]:
new_res = [(words[i], words[j], new_PPMI[i][j]) for j in range(7) for i in range(7)]
sorted(new_res, key=lambda v: v[2], reverse=True)[:10]

[('создание', 'творчество', 10.528136854894825),
 ('создание', 'труд', 6.5149365447868073),
 ('создание', 'дело', 6.1288648217044974),
 ('творчество', 'дело', 5.4402316545090184),
 ('творчество', 'создание', 5.3578918621852187),
 ('творчество', 'труд', 4.4531420725044928),
 ('создание', 'работа', 4.3191422950531617),
 ('творчество', 'деятельность', 3.9056663566093341),
 ('творчество', 'действие', 3.8762572137881746),
 ('работа', 'труд', 3.8226241293890553)]